In [1]:
#     =====================================================================================   #
#                                       FIRST SECTION CODE                                    #
#     =====================================================================================   #

# IMPORTING IMPORTANT MODULE..... 
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd

# IMPORTING ORIGINAL DATA 
jamb_data = pd.read_csv('Newrecord/ibadanfile.csv')

# GETTING ATTRIBUTE FROM ORIGINAL DATA
candidate_name = jamb_data["RG_CANDNAME"].values
candidate_reg = jamb_data["RG_NUM"].values
names = list(candidate_name)
regs = list(candidate_reg)
# print(names)
# print(regs)

# GENERATING NEW DATAFRAME
gen_df = pd.DataFrame()
    
try:
    gen_df = pd.read_csv('Newrecord/ibadan_gen/ibadan_gen1.csv', index=False)
except Exception: 
    pass

def create_dataframe(dataframe,  add=''):
    attributes = []

    for attr in dataframe.columns: 
        attributes.append(attr)
    if add != '':
        attributes.append(add)

    df = pd.DataFrame([] , columns=attributes)

    return df , attributes


if gen_df.empty:
    print('original data is empty')
    gen_df , _ = create_dataframe(jamb_data, 'Contact')

else: 
    pass


original data is empty


In [2]:
#     =====================================================================================   #
#                                HELPER METHOD FOR SECTION TWO                                #
#     =====================================================================================   #




# helper method to valide content compatibility 

def check_content_matching(o_reg='data_reg' , o_name='d_name' , m_reg='mine_reg' , m_name = 'm_name'): 
    
    #   : o_reg / the import jamb student registration number 
    #   : o_name / the import jamb student name 
    #   : m_reg / the mined student registration number 
    #   : m_name / the mined student name 

    validate = False 
    if (m_reg.upper().strip() == o_reg.upper().strip() and m_name.lower() in o_name.lower().split()):
        validate = True
    else : 
        validate = False
    return validate ; 

# helper method to had new row to the new table
def add_row(old_table , new_table, tel='',  m_reg = ''):
    
    if m_reg != '':
        d = old_table[old_table[old_table.columns[0]] == m_reg]
        old_val = d.values
#         print(old_val.flatten())
        temp_value = []
        for value in list(old_val.flatten()):
            temp_value.append(value)

#         print('TEL:' , tel)

        if tel != '':
            temp_value.append(str(tel))
            
            print(temp_value)
            new_table.loc[len(new_table)] = temp_value
            
            new_table.to_csv('Newrecord/ibadan_gen/ibadan_gen1.csv')
            print('add row successfully..........')
            return new_table
                
# add_row(jamb_data, gen_df , tel, reg_number)


In [4]:
#     =====================================================================================   #
#                                       SECOND SECTION CODE                                   #
#     =====================================================================================   #

# eporting to json 

#  to json ...
import json as j
# NAVIGATING THE WEB BROWSER USING THE STUDENT REGISTRATION NUMBER

driver= webdriver.Chrome()

def fetch_info(reg_number):
    driver.get('https://efacility.jamb.gov.ng/PrintExaminationSlip') 
    reg = driver.find_element('id' , "txtRegNumber")
    reg.send_keys(reg_number)

    # submitting form
    # reg.submit()
    sb = driver.find_element('id' , "lnkSearch")

    sb.click()
    # driver.refresh()
    # print('WEB PAGE ', driver.current_url)


    # getting data from html document
    print('fetching content')
    url = f'https://efacility.jamb.gov.ng/TempFiles/Main_Examination_Slip_{reg_number}.htm'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    data = soup.find_all('label')

    return data , len(data)


#  scraping through web for contact 

for index in range(2739, len(regs)): 
    reg = regs[index]
    name = names[index]
      
    
#         with  open('reg_cache.json','w') as file:  
#             file.dump({'last_reg':reg ,  'last_name':name , 'index':index})

    
    print()
    print(f'==== SOURCE FILE SIZE IS ==>  {jamb_data.shape[0]}')
    print('===================================================')
    print(f'   START MINING   FOR :  {reg} : index : {index}  ')
    print('===================================================')

    data , lenght = fetch_info(reg)
    if lenght != 0 :
            
        # printing data for confirmation
        try:
            surname = data[1].get_text()
            firstname = data[2].get_text()
            othername = data[4].get_text()
            tel = data[12].get_text()
            reg_number = data[15].get_text()

            print('Surname : ' , surname)
            print('Fist Name : ', firstname)
            print('Other Name : ',  othername)
            print('Telephone : ' , tel)
            print('Registion Number : ' , reg_number)


            valid = check_content_matching(reg, name , reg_number, surname)
            print('IS VALID ? ', valid)
            if valid :
                print('                     STUDENT CONFIRMED... ')
                add_row(jamb_data, gen_df , tel, reg_number)
                
        except Exception:
            print('error')
                  
    print('==================================================')
    print(f'              END MINING     No.   {index}       ')
    print('==================================================')
        
        
    
            
            

        
print('==================================================')
print(f'            FINAL END OF MINING                  ')
print('==================================================')
        


==== SOURCE FILE SIZE IS ==>  2761
   START MINING   FOR :  202210216548JA : index : 2739  
fetching content
Surname :  Olaniyan
Fist Name :  Kamaldeen
Other Name :  Opeyemi
Telephone :  09157401526
Registion Number :  202210216548JA
IS VALID ?  True
                     STUDENT CONFIRMED... 
['202210216548JA', 'Olaniyan Kamaldeen Opeyemi', 'M', 'OYO', 128, 'Mass Communication', nan, 'Not Admitted', 'OYO', 'THE POLYTECHNIC, IBADAN, OYO STATE', 'IBARAPA-NORTH', '09157401526']
add row successfully..........
              END MINING     No.   2739       

==== SOURCE FILE SIZE IS ==>  2761
   START MINING   FOR :  202211674296DF : index : 2740  
fetching content
Surname :  Akinleye
Fist Name :  Habeeb
Other Name :  Adekunle
Telephone :  09019854416
Registion Number :  202211674296DF
IS VALID ?  True
                     STUDENT CONFIRMED... 
['202211674296DF', 'Akinleye Habeeb Adekunle', 'M', 'OYO', 128, 'Mass Communication', nan, 'Not Admitted', 'OYO', 'THE POLYTECHNIC, IBADAN, OYO STAT

fetching content
Surname :  Akinrinde
Fist Name :  Timileyin
Other Name :  Samuel
Telephone :  07025732690
Registion Number :  202211672598HA
IS VALID ?  True
                     STUDENT CONFIRMED... 
['202211672598HA', 'Akinrinde Timileyin Samuel', 'M', 'OSUN', 121, 'Art & Design', nan, 'Not Admitted', 'OSUN', 'THE POLYTECHNIC, IBADAN, OYO STATE', 'IFE NORTH', '07025732690']
add row successfully..........
              END MINING     No.   2750       

==== SOURCE FILE SIZE IS ==>  2761
   START MINING   FOR :  202210655734IF : index : 2751  
fetching content
Surname :  Nureni
Fist Name :  Abdulrahmon
Other Name :  Adio
Telephone :  07051520184
Registion Number :  202210655734IF
IS VALID ?  True
                     STUDENT CONFIRMED... 
['202210655734IF', 'Nureni Abdulrahmon Adio', 'M', 'OYO', 121, 'Mass Communication', nan, 'Not Admitted', 'OYO', 'THE POLYTECHNIC, IBADAN, OYO STATE', 'IDO', '07051520184']
add row successfully..........
              END MINING     No.   2751       

In [ ]:
# 2182  

In [ ]:
import pandas as pd 
import os

data = pd.read_csv('pool_ekiti/foye_gen1.csv')
data.shape

# def pd_tuple(dataframe):
    
#     for index in range(len(dataframe)):
#         tup_list = tuple(data.iloc[index].values)
        
#         return tup_list 

    